## Dataset

Amod/mental_health_counseling_conversations

In [1]:
# dataset_name = 'KonradSzafer/stackoverflow_python_preprocessed'
# dataset_name = 'graycatHCO3/CodeAlpaca-20K-Python'
dataset_name = 'glaiveai/glaive-code-assistant'

In [2]:
import datasets

dataset = datasets.load_dataset(dataset_name)

print(dataset)

c:\Users\h4has\anaconda3\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 548/548 [00:00<00:00, 182187.59 examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 4777
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 548
    })
})


In [3]:
print(dataset['train'])
print(type(dataset['train']))

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 4777
})
<class 'datasets.arrow_dataset.Dataset'>


In [4]:
data_dict = [data for data in dataset['train']]

In [5]:
print(len(data_dict))

4777


In [6]:
for data in dataset['train']:
    print(data)
    break

{'prompt': 'Write a reusuable function in Python that takes two string variables and returns the longest string.\n', 'completion': 'def longest_string(str1, str2):\n    if len(str1) > len(str2):\n        return str1\n    else:\n        return str2'}


## Model

In [100]:
# model = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
# model = 't5-small'
model = 'flan-t5-small'

In [101]:
# from transformers import LlamaTokenizer, Llama

# tokenizer = LlamaTokenizer.from_pretrained(model)
# model = LlamaForCausalLM.from_pretrained(model)


In [161]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained(model)
model = T5ForConditionalGeneration.from_pretrained(model)

In [103]:
# query = "What is mental health?"
# inputs = tokenizer(query, return_tensors="pt")
# print(inputs)

In [104]:

outputs = model.generate(**inputs)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

c:\Users\h4has\anaconda3\envs\rag\lib\site-packages\transformers\generation\utils.py:1259: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[    0,  8477, 18154,     3,   390,  2869,    54,   199,    25,  1591,
             6,    36,   213,    25,    33,     3,     6,  4839,    11, 16363]])
Mindfulness based practices can help you ground, be where you are, relax and regulate


## Basic Pipeline

In [12]:
import pandas as pd

In [13]:
# df = pd.DataFrame(data_dict)
# df.head(3)
# df.to_csv('data/data.csv', index=False)

In [80]:
df = pd.read_json('ml.json')
print(len(df))
df.head(3)
df.to_csv('data/data.csv', index=False)

639


In [81]:
df.isna().sum()

question    0
answer      0
dtype: int64

In [82]:
df.dropna(inplace=True)

In [17]:
# Retreiving
# Indexing - Elasticsearch, TF-IDF 
# Querying

In [84]:
import minisearch
ms = minisearch.Index(
    text_fields=["question", "answer"],
    keyword_fields=["question"]
)

In [85]:
ms

create index - minsearch

In [86]:
data_dict = df.to_dict(orient='records')

In [87]:
data_dict[0]

{'question': 'What’s the trade-off between bias and variance?',
 'answer': 'Bias is error due to erroneous or overly simplistic assumptions in the learning algorithm, leading to underfitting and poor generalization. Variance is error due to too much complexity in the learning algorithm, leading to overfitting and high sensitivity to training data. The bias-variance decomposition breaks down learning error into bias, variance, and irreducible error. Increasing model complexity reduces bias but increases variance. The goal is to find a balance to minimize total error.'}

In [88]:
ms.fit(data_dict)

search using index

In [89]:
from sentence_transformers import SentenceTransformer

In [93]:
embedding_model = 'multi-qa-MiniLM-L6-cos-v1'

In [94]:
model = SentenceTransformer(embedding_model)
embedding =model.encode('How to sort a dictionary in python?')

In [91]:
print(len(embedding))
print(type(embedding))
print(embedding.tolist())

768
<class 'numpy.ndarray'>
[0.012473829090595245, -0.03423825651407242, 0.02401108294725418, 0.062352072447538376, 0.012287676334381104, 0.04485062137246132, -0.01238344144076109, 0.00038591978955082595, -0.005918867886066437, -0.047173574566841125, 0.01976957358419895, -0.01812436431646347, 0.028216853737831116, 0.005306615494191647, -0.05160516873002052, -0.0032452542800456285, -0.02006971277296543, -0.021828610450029373, 0.06239515170454979, 0.02873624488711357, -0.06761425733566284, -0.028046200051903725, -0.033938150852918625, -0.027862848713994026, 0.022949669510126114, 0.0379955880343914, -0.0070092580281198025, 0.03170578181743622, 0.03905501216650009, -0.05780861899256706, -0.012664245441555977, 0.037202443927526474, -0.0290141012519598, 0.018525587394833565, 1.3947289971838472e-06, -0.033404428511857986, 0.01877940259873867, -0.03210664168000221, 0.044481415301561356, -0.020577382296323776, 0.05126888304948807, 0.039636582136154175, -0.012339640408754349, -0.0291834529489278

In [92]:
print(len(data_dict))

639


#### Vector DB SentenceTransformers

In [98]:
from sentence_transformers import SentencesDataset

model = SentenceTransformer(embedding_model,truncate_dim=128)

In [99]:
# model.encode('How to sort a dictionary in python?')

In [100]:
data_dict[0]

{'question': 'What’s the trade-off between bias and variance?',
 'answer': 'Bias is error due to erroneous or overly simplistic assumptions in the learning algorithm, leading to underfitting and poor generalization. Variance is error due to too much complexity in the learning algorithm, leading to overfitting and high sensitivity to training data. The bias-variance decomposition breaks down learning error into bias, variance, and irreducible error. Increasing model complexity reduces bias but increases variance. The goal is to find a balance to minimize total error.'}

In [101]:
print(len(data_dict))

639


In [102]:
embedded_data = []
for data in data_dict[:10]:
    data['answer_vector'] = model.encode(data['answer'])
    data['question_vector'] = model.encode(data['question'])
    embedded_data.append(data)

In [103]:
print(embedded_data[0])

{'question': 'What’s the trade-off between bias and variance?', 'answer': 'Bias is error due to erroneous or overly simplistic assumptions in the learning algorithm, leading to underfitting and poor generalization. Variance is error due to too much complexity in the learning algorithm, leading to overfitting and high sensitivity to training data. The bias-variance decomposition breaks down learning error into bias, variance, and irreducible error. Increasing model complexity reduces bias but increases variance. The goal is to find a balance to minimize total error.', 'answer_vector': array([ 3.93690355e-02,  1.68576669e-02,  2.18878947e-02, -1.77346300e-02,
        9.37577784e-02, -4.56591864e-04, -1.09980758e-02, -2.95686591e-02,
        3.20731215e-02,  1.96286440e-02, -4.71137872e-04,  2.88731828e-02,
        6.66112732e-03,  1.17804017e-02, -2.79367971e-03,  7.56513625e-02,
       -5.07901087e-02,  4.23743948e-02, -1.05550721e-01, -8.33066702e-02,
       -5.01522049e-02, -5.6640517

#### ElasticSearch

In [104]:
from elasticsearch import Elasticsearch

In [105]:
es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '7d50c3ad219e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'GkHNY1cCQQGIGcngnIggvA', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [115]:
index_settings={
    "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
        "question": {"type": "text"},
        "answer": {"type": "text"},
        "answer_vector": {"type": "dense_vector", "dims": 128, "index": True, "similarity": "cosine"},
        "question_vector": {"type": "dense_vector", "dims": 128, "index": True, "similarity": "cosine"},
        }
    }
}
index_name = 'python-qa-index'

query='What is the tradeoff between bias and variance?'

In [116]:
es.indices.delete(index=index_name,ignore_unavailable=True)
es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-qa-index'})

In [117]:
query_vector = model.encode(query)

In [110]:
knn_query = {
    "field": "answer_vector",
    "query_vector": query_vector,
    "k": 5,
    "num_candidates": 5
}

In [113]:
results = es.search(index=index_name, knn=knn_query, size = 5)

In [118]:
results

ObjectApiResponse({'took': 11, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [114]:
results['hits']['hits']

[]

In [63]:
for doc in embedded_data:
    es.index(index=index_name, document=doc)

In [46]:
es

<Elasticsearch(['http://localhost:9200'])>

#### Generate Response

In [27]:
# query = "How to create a dictionary in python?"
query = "How to sort a dictionary in python?"

In [28]:
boost = {'question': 3.0, 'section': 0.5}

results = ms.search(
    query=query,
    # filter_dict={'course': 'data-engineering-zoomcamp'},
    # boost_dict=boost,
    num_results=2
)

In [29]:
answers = [result['answer'] for result in results]
answers

['Certainly! In Python, you can sort a dictionary based on its values using the `sorted()` function along with the `items()` method of the dictionary. Let me explain the solution to you.\n\nHere\'s an example code that sorts a dictionary based on its values in ascending order:\n\n```python\ndictionary = {"a": 10, "b": 3, "c": 4, "d": 1, "e": 5}\nsorted_dict = dict(sorted(dictionary.items(), key=lambda x: x[1], reverse=False))\n\nprint("Sorting Dictionary as per the Value:")\nprint("The Original Dictionary:", dictionary)\nprint("The Sorted Dictionary:", sorted_dict)\n```\n\nExplanation:\n\n1. We first define a dictionary called `dictionary` with key-value pairs representing some data.\n\n2. Then, we use the `sorted()` function to sort the dictionary based on its values. The `items()` method helps to iterate over the key-value pairs of the dictionary.\n\n3. The `key` parameter in the `sorted()` function takes a lambda function `lambda x: x[1]`, which specifies that we want to sort based 

In [177]:
prompt_template = """
You're a python coder. Provide concise and short python code for the question based on the context.
QUESTION: {question}

CONTEXT: 
{response}
If you dont know the answer, just say that you dont know.
""".strip()

In [178]:
results[0]

{'title': 'Sort a Python dictionary by value',
 'answer': 'Why not try this approach. Let us define a dictionary called mydict with the following data:\nmydict = {\'carl\':40,\n          \'alan\':2,\n          \'bob\':1,\n          \'danny\':3}\n\nIf one wanted to sort the dictionary by keys, one could do something like:\nfor key in sorted(mydict.iterkeys()):\n    print "%s: %s" % (key, mydict[key])\n\nThis should return the following output:\nalan: 2\nbob: 1\ncarl: 40\ndanny: 3\n\nOn the other hand, if one wanted to sort a dictionary by value (as is asked in the question), one could do the following:\nfor key, value in sorted(mydict.iteritems(), key=lambda (k,v): (v,k)):\n    print "%s: %s" % (key, value)\n\nThe result of this command (sorting the dictionary by value) should return the following:\nbob: 1\nalan: 2\ndanny: 3\ncarl: 40\n\n',
 'question': 'I have a dictionary of values read from two fields in a database: a string field and a numeric field. The string field is unique, so t

In [179]:
prompt = prompt_template.format(question=query, response=answers)

In [180]:
prompt

'You\'re a python coder. Provide concise and short python code for the question based on the context.\nQUESTION: How to sort a dictionary in python?\n\nCONTEXT: \n[\'Why not try this approach. Let us define a dictionary called mydict with the following data:\\nmydict = {\\\'carl\\\':40,\\n          \\\'alan\\\':2,\\n          \\\'bob\\\':1,\\n          \\\'danny\\\':3}\\n\\nIf one wanted to sort the dictionary by keys, one could do something like:\\nfor key in sorted(mydict.iterkeys()):\\n    print "%s: %s" % (key, mydict[key])\\n\\nThis should return the following output:\\nalan: 2\\nbob: 1\\ncarl: 40\\ndanny: 3\\n\\nOn the other hand, if one wanted to sort a dictionary by value (as is asked in the question), one could do the following:\\nfor key, value in sorted(mydict.iteritems(), key=lambda (k,v): (v,k)):\\n    print "%s: %s" % (key, value)\\n\\nThe result of this command (sorting the dictionary by value) should return the following:\\nbob: 1\\nalan: 2\\ndanny: 3\\ncarl: 40\\n\\n\'

In [181]:
len(prompt)

1591

In [182]:
inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True, padding='max_length')
print(inputs)

{'input_ids': tensor([[  148,    31,    60,     3,     9,     3,   102,    63,   189,   106,
          1081,    52,     5,  7740, 22874,    11,   710,     3,   102,    63,
           189,   106,  1081,    21,     8,   822,     3,   390,    30,     8,
          2625,     5,     3, 15367,   134,  9562,    10,   571,    12,  1843,
             3,     9, 24297,    16,     3,   102,    63,   189,   106,    58,
          8472,  3463,     4,   382,    10,   784,    31, 17891,    59,   653,
            48,  1295,     5,  1563,   178,  6634,     3,     9, 24297,   718,
            82, 12194,    28,     8,   826,   331,    10,     2,    29,  2258,
         12194,  3274,     3,     2,    31,  1720,    40,     2,    31,    10,
          2445,     6,     2,    29,     3,     2,    31,     9,  1618,     2,
            31,    10,  4482,     2,    29,     3,     2,    31, 17396,     2,
            31,    10,  4347,     2,    29,     3,     2,    31,    26, 15159,
             2,    31,    10,   519,  

In [183]:
outputs = model.generate(
    inputs['input_ids'], 
    max_length=150, 
    num_beams=4, 
    # early_stopping=True
)
print(outputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tensor([[    0,     3, 15367,   134,  9562,    10,   784,    31, 17891,    59,
           653,    48,  1295,     5,  1563,   178,  6634,     3,     9, 24297,
           718,    82, 12194,    28,     8,   826,   331,    10,     2,    29,
          2258, 12194,  3274,     3,     2,    31,  1720,    40,     2,    31,
            10,  2445,     6,     2,    29,     3,     2,    31,     9,  1618,
             2,    31,    10,  4482,     2,    29,     3,     2,    31, 17396,
             2,    31,    10,  4347,     2,    29,     3,     2,    31,    26,
         15159,     2,    31,    10,   519,     2,    29,     2,    29,   634,
           741,    13,    48,  4106,    41,  9309,    53,     8, 24297,    57,
           701,    61,   225,  1205,     8,   826,  3911,    10,     2,    29,
         17396,    10,   209,     2,    29,     9,  1618,    10,   204,     2,
           727, 15159,    10,   220,     1]])
QUESTION: ['Why not try this approach. Let us define a dictionary called mydict with 

### Handling longer chunks


## Basic Pipeline

In [ ]:
# Retreiving
# Indexing - Elasticsearch, TF-IDF 
# Querying

In [ ]:
import minisearch


ms = minisearch.Index(
    text_fields=["Response"],
    keyword_fields=["Context"],
)

In [ ]:
ms

minsearch

In [ ]:
ms.fit(data_dict)

In [ ]:
def split_text(text, max_length=512):
    # Split text into chunks of max_length tokens
    tokens = tokenizer(text, return_tensors="pt")['input_ids'][0]
    chunks = [tokens[i:i+max_length] for i in range(0, len(tokens), max_length)]
    return chunks

# Split long text
text_chunks = split_text(long_text)

# Generate output for each chunk
results = []
for chunk in text_chunks:
    inputs = tokenizer.decode(chunk, return_tensors="pt").unsqueeze(0)
    outputs = model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    results.append(decoded_output)

# Combine results if necessary
combined_output = ' '.join(results)
print(combined_output)


## End